# Zoomcamp MLOps Course Cohort 2024 Week 01 Homework
*Version:* `1.2` *(Jupytext, time measurements, logger, param notebook execution, fixes)*

<a name="ToC"></a>
# Table of Content

- [Notebook Description](#0)
- [General Settings](#1)
    - [Paths](#1-1)
    - [Notebook Functionality and Appearance](#1-2)
    - [External Libraries](#1-3)
    - [Internal Code](#1-4)
    - [Constants](#1-5)   
- [Analysis](#2)   
    - [Data Reading](#2-1)   
    - [Questions](#2-2)
        - [Q1 Read the data for January. How many columns are there?](#2-2-1)     
        - [Q2 Computing duration](#2-2-2)
        - [](#2-2-3)
        - [](#2-2-4)
        - [](#2-2-5)
        - [](#2-2-6)
- [Final Timestamp](#3)  

<a name="0"></a>
# Notebook Description
[ToC](#ToC) 

> *Please put your comments about the notebook functionality here.*  

<a name="1"></a>
# GENERAL SETTINGS
[ToC](#ToC)  
General settings for the notebook (paths, python libraries, own code, notebook constants). 

> *NOTE: All imports and constants for the notebook settings shoud be here. Nothing should be imported in the analysis section.*

<a name="1-1"></a>
### Paths
[ToC](#ToC)  

Adding paths that are necessary to import code from within the repository.

In [1]:
import sys
import os
sys.path+=[os.path.join(os.getcwd(), ".."), os.path.join(os.getcwd(), "../..")] # one and two up

<a name="1-2"></a>
### Notebook Functionality and Appearance
[ToC](#ToC)  
Necessary libraries for notebook functionality:
- A button for hiding/showing the code. By default it is deactivated and can be activated by setting CREATE_BUTTON constant to True. 
> **NOTE: This way, using the function, the button works only in active notebook. If the functionality needs to be preserved in html export, then the code has to be incluced directly into notebook.**
- Set notebook width to 100%.
- Notebook data frame setting for better visibility.
- Initial timestamp setting and logging the start of the execution.

#### Overall Setting Specification

In [2]:
LOGGER_CONFIG_NAME = "logger_file_limit_console"
ADDAPT_WIDTH = True

#### Overall Behaviour Setting

In [3]:
try:
    from src.utils.notebook_support_functions import create_button, get_notebook_name
    NOTEBOOK_NAME = get_notebook_name()
    SUPPORT_FUNCTIONS_READ = True
except:
    NOTEBOOK_NAME = "NO_NAME"
    SUPPORT_FUNCTIONS_READ = False  

In [4]:
from src.utils.logger import Logger
from src.utils.envs import Envs
from src.utils.config import Config
from pandas import options
from IPython.display import display, HTML

In [5]:
options.display.max_rows = 500
options.display.max_columns = 500
envs = Envs()
envs.set_logger(LOGGER_CONFIG_NAME)
Logger().start_timer(f"NOTEBOOK; Notebook name: {NOTEBOOK_NAME}")
if ADDAPT_WIDTH:
    display(HTML("<style>.container { width:100% !important; }</style>")) # notebook width

In [6]:
# create_button()

<a name="1-3"></a>
### External Libraries
[ToC](#ToC)  

In [7]:
from datetime import datetime

from os.path import join

import pandas as pd

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

from IPython.display import display, HTML

<a name="1-4"></a>
### Internal Code
[ToC](#ToC)  
Code, libraries, classes, functions from within the repository.

In [36]:
from src.utils.date_time_functions import create_datetime_id

<a name="1-5"></a>
### Constants
[ToC](#ToC)  
Constants for the notebook.

> *NOTE: Please use all letters upper.*

#### General Constants
[ToC](#ToC)  

In [9]:
# from src.global_constants import *  # Remember to import only the constants in use
N_ROWS_TO_DISPLAY = 2
FIGURE_SIZE_SETTING = {"autosize": False, "width": 2200, "height": 750}
DATA_PROCESSING_CONFIG_NAME = "data_processing_basic"

#### Constants for Setting Automatic Run
[ToC](#ToC)  

In [10]:
# MANDATORY FOR CONFIG DEFINITION AND NOTEBOOK AND ITS OUTPUTS IDENTIFICATION #########################################
PYTHON_CONFIG_NAME = "python_local"
ID = create_datetime_id(now=datetime.now(), add_micro=False)
# (END) MANDATORY FOR CONFIG DEFINITION AND NOTEBOOK AND ITS OUTPUTS IDENTIFICATION ###################################

#### Python Config Initialisation
[ToC](#ToC)  

In [11]:
envs.set_config(PYTHON_CONFIG_NAME)

#### Notebook Specific Constants
[ToC](#ToC)  

<a name="2"></a>
# ANALYSIS
[ToC](#ToC)  

In [41]:
def print_info_about_data_frame(df: pd.DataFrame) -> None:
    """
    Print overall df stats.
    :param df: DataFrame. Data to have its stats printed.
    """
    print(f"DataFrame type: {str(type(df))}")
    print(f"DataFrame shape: {str(df.shape)}")
    print(f"DataFrame dtypes: {df.dtypes.apply(lambda x: x.name).to_dict()}")
    print("\n")
    print(f"DataFrame head:\n{df.head()}")
    print("\n")
    print(f"DataFrame description:\n{df.describe()}")
    print("\n")

<a name="2-1"></a>
## Data Reading
[ToC](#ToC)  

In [12]:
file_name = "yellow_tripdata_2023-01.parquet"
df_january = pd.read_parquet(join(Config().get_data().path.external_data, file_name))
file_name = "yellow_tripdata_2023-02.parquet"
df_february = pd.read_parquet(join(Config().get_data().path.external_data, file_name))

In [13]:
df_january.shape

In [14]:
df_january.shape

In [15]:
df_january.head()

<a name="2-2"></a>
## Questions
[ToC](#ToC)  

<a name="2-2-1"></a>
### Q1 Read the data for January. How many columns are there?
[ToC](#ToC)  

In [16]:
answer = f"The number of Columns is: {df_january.shape[1]}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-2"></a>
### Q2 Computing duration
[ToC](#ToC)  

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [17]:
ATTR_DURATION = "DURATION"

In [18]:
def add_duration_attribute(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds duration attribute to a data frame.
    """
    df[ATTR_DURATION] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df[ATTR_DURATION] = df[ATTR_DURATION].apply(lambda td: td.total_seconds() / 60)
    
    return df

In [19]:
df_january = add_duration_attribute(df_january)
df_february = add_duration_attribute(df_february)

In [20]:
df_january[[ATTR_DURATION]].head()

In [21]:
avg_duration = round(df_january[ATTR_DURATION].std(), 2)
print(avg_duration)

In [22]:
answer = f"The standard deviation of the trips duration in January is: {avg_duration}"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-3"></a>
### Q3 Dropping outliers
[ToC](#ToC)  

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [23]:
def remove_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes outliers from data frame.
    """
    in_between = [1. <= duration <= 60. for duration in df[ATTR_DURATION]]
    
    original_size = df.shape[0]
    df = df.loc[in_between, ]
    
    print(f"DF N of Obs: {original_size}")
    print(f"N of In Between: {sum(in_between)}")
    print(f"New df size    : {df.shape[0]}")
    print(f"Franction: {round(df.shape[0] / original_size, 4)}")
       
    return df, original_size  

In [24]:
df_january, orig_size_january = remove_outliers(df_january)
df_february, orig_size_february = remove_outliers(df_february)

In [25]:
observations_franction_left = round((df_january.shape[0] + df_february.shape[0]) / (orig_size_january + orig_size_february), 4)
observations_franction_left

In [29]:
answer = f"The number of observations left is: {round(observations_franction_left * 100, 2)} %"
display(HTML(f"<p style='font-weight:bold; background-color:black; color:white; font-size:20px; padding:10px;'>Answer: {answer}</p>"))

<a name="2-2-4"></a>
### Q4 One-hot encoding
[ToC](#ToC)  

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

In [28]:
df = df_january.copy()

In [31]:
ATTR_PU_LOC_ID = "PULocationID"
ATTR_DO_LOC_ID = "DOLocationID"

In [30]:
df.head()

In [42]:
print_info_about_data_frame(df[[ATTR_PU_LOC_ID, ATTR_DO_LOC_ID]])

In [46]:
df = df[[ATTR_PU_LOC_ID, ATTR_DO_LOC_ID]].astype(str)

In [48]:
df.dtypes

<a name="2-2-5"></a>
### Q5 Training a model
[ToC](#ToC)  

<a name="2-2-6"></a>
### Q6 Evaluating the model
[ToC](#ToC)  

<a name="3"></a>
# Final Timestamp
[ToC](#ToC)  

In [27]:
Logger().end_timer()